In [1]:
from Camera import Camera
import numpy as np
from Frame import Frame

import matplotlib.pyplot as plt
from GenerateGsInput import GenerateGsInput
import Plotters
import plotly.graph_objects as go
import scipy.io
import pandas as pd
import Utils

%matplotlib qt

# load hull
path = 'G:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}


In [2]:
# initilize objects
frames = list(range(500,700,13))

image_name,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {im_name:Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_name)}

In [3]:
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)

ggs = GenerateGsInput(path,frames)
ggs.save_croped_images()
ggs.generate_camera_text(croped_image = True)
ggs.generate_image_text()
ggs.generate_points3d_text(voxel_dict,colors_dict)



In [14]:
# plots
# camera ptcloud
color_pts = ['red','green','blue','black']
fig = go.Figure()
im_name = list(frames.keys())[0]
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    Plotters.scatter3d(fig,frames[image].voxels_with_idx,color_pts[cam],3)
fig.show()



# projection
fig,axs = plt.subplots(2,2)
for cam in range(4):
    image = f'{im_name.split("CAM")[0]}CAM{cam+1}.jpg'
    proj = frames[image].project_on_image(frames[image].points_in_ew_frame_homo,croped_camera_matrix = True)
    axs[cam // 2,cam % 2].imshow(frames[image].croped_image)
    axs[cam // 2,cam % 2].scatter(proj[:,0],proj[:,1],s = 3,c = 'red')


In [ ]:
from colmap_loader import read_extrinsics_text, read_intrinsics_text, qvec2rotmat, \
    read_extrinsics_binary, read_intrinsics_binary, read_points3D_binary, read_points3D_text


cameras_extrinsic_file = f'{ggs.sparse_dir}/images.txt'
cameras_intrinsic_file = f'{ggs.sparse_dir}/cameras.txt'
txt_path =f'{ggs.sparse_dir}/points3D.txt'

xyz, rgb, _ = read_points3D_text(txt_path)

cam_extrinsics = read_extrinsics_text(cameras_extrinsic_file)
cam_intrinsics = read_intrinsics_text(cameras_intrinsic_file)
